In [1]:
import xmlrpc.client as xmlrpclib

cmd = xmlrpclib.ServerProxy('http://localhost:9123')
cmd.fragment('ala')

In [3]:
cmd.delete('all')

In [2]:
cmd.delete('all')
cmd.do('load ./pdb/6bd0.pdb, multiplex=1; as sticks')

In [10]:
input = [
    ("11013331", "KAT"),
    ("9085267", "NOT"),
    ("5238761", "ETH"),
    ("5349618", "ETH"),
    ("11788544", "NOT"),
    ("962142", "ETH"),
    ("7795297", "ETH"),
    ("7341464", "ETH"),
    ("9843236", "KAT"),
    ("5594916", "ETH"),
    ("1550003", "ETH"),
]
from collections import defaultdict

res = defaultdict(list)
for v, k in input:
    res[k].append(v)

print(input)
print(res)

[('11013331', 'KAT'), ('9085267', 'NOT'), ('5238761', 'ETH'), ('5349618', 'ETH'), ('11788544', 'NOT'), ('962142', 'ETH'), ('7795297', 'ETH'), ('7341464', 'ETH'), ('9843236', 'KAT'), ('5594916', 'ETH'), ('1550003', 'ETH')]
defaultdict(<class 'list'>, {'KAT': ['11013331', '9843236'], 'NOT': ['9085267', '11788544'], 'ETH': ['5238761', '5349618', '962142', '7795297', '7341464', '5594916', '1550003']})


In [114]:
import time

cmd.delete('all')
time.sleep(0.700)
cmd.delete('all')

cmd.do('load ./pdb/5mct.pdb, multiplex=1; as sticks')
# cmd.do('load ./pdb/5mct.pdb; as sticks')
# cmd.do('load ./pdb/5mct.pdb, multiplex=1')

In [117]:
cmd.delete('all')
cmd.do('load ./pdb/1ytb.pdb, multiplex=1; as sticks')

In [116]:
cmd.do('''
color grey, polymer.protein
color white, polymer.nucleic
''')

In [119]:
# cmd.do('''
# list=[]
# iterate (boo),list.append((resi + "/" +chain + "/" +resn))
# print list
# ''')

myspace = {'bfactors': []}
cmd.iterate('(boo)', 'bfactors.append("123")', 1, myspace)
# cmd.iterate('(boo)', 'bfactors.append(("///" + chain + "/" + resn + "`" + resi + "/"))', 1, myspace)
print(myspace)
print(myspace['bfactors'])
# cmd.iterate('(boo)', 'bfactors.append(b)')


{'bfactors': []}
[]


In [4]:
import os
import glob
import sys
import pandas
import time

report_dir = "../reports/"
#
list_of_reports = [
    item for item in os.listdir(report_dir) if "LAGLIDADG_1_6BCE" in item
]
# list_of_reports = [item for item in os.listdir(report_dir) if "report_LAGLIDADG_1_1T9I"  in item]
# list_of_reports = [item for item in os.listdir(report_dir)]

# list_of_reports = [item for item in os.listdir(report_dir) if "binary_table" in item]


for filename in list_of_reports:
    print("filename " + filename)
    report_name = filename.replace(".csv", "")
    for hgx in glob.glob(f"*pymol_rpc_workaround_{report_name}*"):
        os.remove(hgx)
    for hgx in glob.glob(f"sur_{report_name}*.txt"):
        os.remove(hgx)
    df = pandas.read_csv(report_dir + filename)
    # df = pandas.read_csv(report_dir + filename, skiprows=lambda x: df.at[x, 'selector'] == "Protein_AC")
    # print(df)
    # new_df = df.query('selector != Protein_AC')
    # new_df = df.loc[df['selector'] != "Protein_AC"]
    # print(new_df)
    pdb_name = df["selector"][3].split("//")[0].replace("/", "")
    print(pdb_name)

    list_of_selectors = []
    for row in range(len(df)):
        if df.at[row, "selector"] == "Protein_AC":
            continue

        list_of_selectors.append(df.at[row, "selector"])
        # print(df.at[row, 'c1'], df.at[row, 'c2'])
    list_of_selectors_as_str = " or ".join(list_of_selectors)
    list_of_selectors_as_str = list_of_selectors_as_str.replace(pdb_name, "")
    print(list_of_selectors)
    print(list_of_selectors_as_str)
    cmd.delete("all")
    time.sleep(0.900)
    cmd.delete("all")

    cmd.do(f"load ./pdb/{pdb_name}.pdb, multiplex=1; as sticks")

    time.sleep(0.900)
    cmd.do(
        f"""
color grey, polymer.protein
color white, polymer.nucleic
"""
    )
    time.sleep(0.900)
    cmd.do(
        f"""
select cons, ({list_of_selectors_as_str})
"""
    )

    time.sleep(0.900)
    cmd.do(
        f"""
select boo, cons near_to 3.5 of polymer.protein
select foo, boo near_to 3.5 of polymer.nucleic

list=[]
iterate (foo),list.append(("///" + chain + "/" + resn + "`" + resi + "/"))
print list

python
with open("./scripts-src/pymol_resp/pymol_rpc_workaround_{report_name}.txt", "w") as f:
   f.write("\\n".join(list))
f.close()
python end
"""
    )
    time.sleep(0.900)
    cmd.do(
        f"""
select sur, * near_to 3.5 of foo

list_of_sur=[]
iterate (sur),list_of_sur.append(("///" + chain + "/" + resn + "`" + resi + "/"))
print list_of_sur

python
with open("./scripts-src/pymol_resp/sur_{report_name}.txt", "w") as f:
   f.write("\\n".join(list_of_sur))
f.close()
python end
"""
    )
    time.sleep(5.900)

    # print(df.head())

filename report_LAGLIDADG_1_6BCE.csv
6bce
['/6bce//C/HOH`131/O', '/6bce//C/HOH`130/O', '/6bce//C/HOH`129/O', '/6bce//C/HOH`123/O', '/6bce//C/HOH`120/O', '/6bce//C/HOH`110/O', '/6bce//C/HOH`104/O', '/6bce//B/HOH`137/O', '/6bce//B/HOH`135/O', '/6bce//B/HOH`133/O', '/6bce//B/HOH`131/O', '/6bce//B/HOH`128/O', '/6bce//B/HOH`126/O', '/6bce//B/HOH`123/O', '/6bce//B/HOH`122/O', '/6bce//B/HOH`118/O', '/6bce//B/HOH`117/O', '/6bce//B/HOH`116/O', '/6bce//B/HOH`115/O', '/6bce//B/HOH`114/O', '/6bce//B/HOH`108/O', '/6bce//B/HOH`107/O', '/6bce//A/HOH`658/O', '/6bce//A/HOH`639/O', '/6bce//A/HOH`631/O', '/6bce//A/HOH`593/O', '/6bce//A/HOH`584/O', '/6bce//A/HOH`581/O', '/6bce//A/HOH`573/O', '/6bce//A/HOH`564/O', '/6bce//A/HOH`563/O', '/6bce//A/HOH`555/O', '/6bce//A/HOH`553/O', '/6bce//A/HOH`542/O', '/6bce//A/HOH`541/O', '/6bce//A/HOH`536/O', '/6bce//A/HOH`527/O', '/6bce//A/HOH`526/O', '/6bce//A/HOH`525/O', '/6bce//A/HOH`518/O', '/6bce//A/HOH`517/O', '/6bce//A/HOH`685/O', '/6bce//A/HOH`684/O', '/6bce//A/H